<a href="https://colab.research.google.com/github/uluumy/Fondements-de-l-apprentissage-profond/blob/master/Enonce_Projet_ChatsChiens_Etape3_ApprentissageParTransfert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet: Chats vs Chiens
## Classification d'images: utilisation de modèle pré-entrainé (apprentissage par transfert)

##Étape 1: Mise en place de l'environnement


### Colab

Execution -> Reinitialiser tous les environnemetns d'execussion (lRuntime -> Reset all runtimes)

Execussion -> Modifier le type d'execussion -> Accelereateur materiel -> GPU


### Librairies à importer

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os  ## Pour lire les fichiers et la structure des répertoires
import matplotlib.pyplot as plt
import numpy as np

In [0]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
tf.logging.set_verbosity(tf.logging.ERROR)

### Fonctions utiles

In [0]:
def plot_history(x,y1,y2,y1_label, y2_label, x_label, y_label,title):
  plt.clf()
  plt.plot(x, y1, label=y1_label) 
  plt.plot(x, y2, label=y2_label)
  
  plt.title(title)
  plt.xlabel(x_label)
  plt.ylabel(y_label)
  plt.legend()
  plt.show

In [0]:
# CEtte fonction affiche l'image.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

### --- TODO --- Instantiation du modele pre-entraine (VGG16)
Appeler la methode VGG16. Choisir 'imagenet' comme weights, include_top = False
ET bien choisir le parametre input_shape = (???,???,?)

In [0]:
from keras.applications import VGG16

conv_base = # --- TODO --- ajouter le code ici

In [0]:

conv_base.summary()

## Étape 2: Obtention des données

### Importer

In [0]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
zip_dir = tf.keras.utils.get_file('cats_and_dogs_filterted.zip', origin=_URL, extract=True)

In [0]:
zip_dir_base = os.path.dirname(zip_dir)
!find $zip_dir_base -type d -print

In [0]:
base_dir = os.path.join(os.path.dirname(zip_dir), 'cats_and_dogs_filtered')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats')  # directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')  # directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')  # directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')  # directory with our validation dog pictures

## Étape 3: Extraction des features

In [0]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels

train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)

###Avoir les bonnes dimensions

In [0]:
train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))

## Étape 4: Création du modèle

### Définir le modèle

In [0]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

### Compiler le modèle

In [0]:
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])

### Sommaire du modèle

In [0]:
model.summary()

### Entrainer le modèle

In [0]:
history = model.fit(train_features, train_labels,
                    epochs=30,
                    batch_size=20,
                    validation_data=(validation_features, validation_labels))

### Visualisation des résultats

In [0]:

acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']

epochs_range = range(30)
val_loss = history.history['val_loss']
plot_history(epochs_range, acc, val_acc, 'Training accuracy', 'Validation accuracy', 'Epochs', 'Accuracy','Training and validation Accuracy')

ressources
https://github.com/uluumy/deep-learning-with-python-notebooks/blob/master/5.3-using-a-pretrained-convnet.ipynb


